In [78]:
import base64
import requests
import json
import numpy as np 
import pandas as pd
from tqdm import tqdm
import multiprocessing
import json
import os

from requests.structures import CaseInsensitiveDict

API_KEY = "your-api-key"

def detect(f):
    url = "https://anysee.ap-northeast-1.cloud.japancv.co.jp/api/v1/entities/faces/detect"

    with open(f, 'rb') as f:
        content = f.read()

    encoded_content = base64.b64encode(content).decode('utf-8')
    
    payload = {
        "model": "JCV_FACE_K25000",
        "image": {
            "data": encoded_content,
            "autoRotate": False,
            "returnDetails": {
                "position": True,
                "angle": True,
                "landmarks": False,
                "quality": True,
                "attributes": True
            }
        }
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "api-key": API_KEY
    }
    
    response = requests.post(url, json=payload, headers=headers)
    return response.json()

def quality_check(angle, quality, attributes):
    is_success = True
    is_success &= (angle['yaw']>=-15*2) & (angle['yaw']<=15*2)
    is_success &= (angle['pitch']>=-20*2) & (angle['pitch']<=20*2)
    is_success &= (angle['roll']>=-15*2) & (angle['roll']<=15*2)

    #is_success &= (quality['brightness']>=-0.5) & (quality['brightness']<=0.5)
    is_success &= (quality['sharpness']>=0.8)
    is_success &= (quality['mouthClosed']>=0.6)
    is_success &= (quality['size']>=0) & (quality['size']<=0.85)
    is_success &= (quality['integrity']==1)
    
    is_success &= (quality['completeness']['total']>=0.9)
    is_success &= (quality['completeness']['mouth']>=0.9)
    is_success &= (quality['completeness']['leftEye']>=0.9)
    is_success &= (quality['completeness']['rightEye']>=0.9)
    is_success &= (quality['completeness']['leftEyeBrow']>=0.8)
    is_success &= (quality['completeness']['rightEyeBrow']>=0.8)
    is_success &= (quality['completeness']['nose']>=0.9)
    is_success &= (quality['completeness']['faceContour']>=0.9)
    is_success &= (quality['completeness']['faceContour']>=0.9)
    is_success &= (quality['completeness']['faceContour']>=0.9)
    is_success &= (quality['completeness']['faceContour']>=0.9)
    is_success &= (attributes['mask']['value']=="without_mask")
    is_success &= (attributes['hat']['value']=="without_hat")
    is_success &= (attributes['glasses']['value']!="with_sunglasses")
    return is_success
    
def create_identify(f):
    
    url = "https://anysee.ap-northeast-1.cloud.japancv.co.jp/api/v1/entities/faces"

    with open(f, 'rb') as f:
        content = f.read()

    encoded_content = base64.b64encode(content).decode('utf-8')
    
    payload = {
        "model": "JCV_FACE_K25000",
        "ttl": -1,
        "image": {
            "data": encoded_content,
            "autoRotate": False,
            "returnDetails": {
                "position": False,
                "angle": False,
                "landmarks": False,
                "quality": False,
                "attributes": False
            }
        }
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "api-key": API_KEY
    }
    
    response = requests.post(url, json=payload, headers=headers)
    
    return response.json()

    
def search(f, area):
    #print(url)
    
    #f = f.replace('/home/MS-Celeb-1M/', '')
    
    #port = 10052
    #f = file_path
    
    with open(f, 'rb') as f:
        content = f.read()

    encoded_content = base64.b64encode(content).decode('utf-8')
    url = "https://anysee.ap-northeast-1.cloud.japancv.co.jp/api/v1/entities/faces/search"
    
    payload = {
        "model": "JCV_FACE_K25000",
        "image": {
            "autoRotate": False,
            "returnDetails": {
                "position": True,
                "angle": False,
                "landmarks": False,
                "quality": False,
                "attributes": False
            },
            "data": encoded_content,
            "area": area
        },
        "maxEntities": 1,
        "threshold": 0.9
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "api-key": API_KEY
    }
    
    response = requests.post(url, json=payload, headers=headers)
    
    return response.json()

In [79]:
# get faces with detail information
resp = detect('andrew_ng_register.jpeg')
resp

{'model': 'JCV_FACE_K25000',
 'count': 1,
 'entities': [{'details': {'position': {'top': 189,
     'left': 240,
     'width': 227,
     'height': 263},
    'angle': {'yaw': -12.473504,
     'pitch': -0.8132769,
     'roll': 0.8799506,
     'centerX': 353,
     'centerY': 320},
    'quality': {'brightness': 0.56078434,
     'sharpness': 0.9984498,
     'mouthClosed': 0.6392468,
     'centered': 0.96349365,
     'size': 0.12833047,
     'integrity': 1,
     'completeness': {'total': 0.98248786,
      'leftEyeBrow': 0.993383,
      'rightEyeBrow': 0.97328025,
      'leftEye': 0.9988978,
      'rightEye': 0.9996236,
      'nose': 0.99954957,
      'mouth': 0.9989473,
      'faceContour': 0.9561821}},
    'attributes': {'age': {'value': 41, 'upperLimit': 46, 'lowerLimit': 36},
     'gender': {'value': 'male', 'certainty': 0.9999839067459106},
     'bangs': {'value': 'without_bangs', 'certainty': 0.9999978542327881},
     'facialHair': {'value': 'without_facial_hair',
      'certainty': 0.99

In [80]:
# use 1st entity as an example to register
# but we need to check quality first
entity_detail = resp['entities'][0]['details']
quality_check(entity_detail['angle'], entity_detail['quality'], entity_detail['attributes'])

True

In [81]:
# since quality passed, we could further register this photo
resp = create_identify('andrew_ng_register.jpeg')
resp

{'uuid': '35825d6b-1777-4c30-8360-c9f96fd7d27d',
 'model': 'JCV_FACE_K25000',
 'collection': '',
 'key': '',
 'metadata': [],
 'createdAt': '2023-08-03T03:50:04Z',
 'updatedAt': '2023-08-03T03:50:04Z',
 'expiresAt': '3000-01-01T00:00:00Z'}

In [82]:
# detect all the faces
resp = detect('demo_test.jpeg')
resp

{'model': 'JCV_FACE_K25000',
 'count': 3,
 'entities': [{'details': {'position': {'top': 424,
     'left': 84,
     'width': 436,
     'height': 413},
    'angle': {'yaw': -15.369961,
     'pitch': 6.5996795,
     'roll': -2.1522954,
     'centerX': 302,
     'centerY': 630},
    'quality': {'brightness': 0.18431373,
     'sharpness': 0.9992035,
     'mouthClosed': 0.9997161,
     'centered': 0.5730157,
     'size': 0.09246032,
     'integrity': 1,
     'completeness': {'total': 0.9540124,
      'leftEyeBrow': 0.9252408,
      'rightEyeBrow': 0.9551557,
      'leftEye': 0.99915665,
      'rightEye': 0.9996586,
      'nose': 0.9997987,
      'mouth': 0.99980086,
      'faceContour': 0.89099884}},
    'attributes': {'age': {'value': 25, 'upperLimit': 30, 'lowerLimit': 20},
     'gender': {'value': 'male', 'certainty': 0.9998440146446228},
     'bangs': {'value': 'with_bangs', 'certainty': 0.999500572681427},
     'facialHair': {'value': 'without_facial_hair',
      'certainty': 0.9990276

In [87]:
for entity in resp['entities']:
    position = entity['details']['position']

    # increase the bbox a bit

    t,l,w,h = position['top'], position['left'], position['width'], position['height']
    t -= h//2
    l -= w//2
    t = max(t, 0)
    l = max(l, 0)
    w *= 2
    h *= 2
    
    res = search('demo_test.jpeg', area={"top": t, "left": l, "width": w, "height": h})
    

    if res['count'] == 1:
        print('!!!!!!!!!!!!!!!!!!!!!!!')
        print('entity found!!!!!!!!!!!')
        print(res['entitiesFound'])
        print('!!!!!!!!!!!!!!!!!!!!!!!')
    else:
        print(res)

{'model': 'JCV_FACE_K25000', 'count': 0, 'maxEntities': 1, 'threshold': 0.9, 'entitiesFound': [], 'entityUploaded': {'details': {'position': {'top': 420, 'left': 78, 'width': 456, 'height': 430}}}}
!!!!!!!!!!!!!!!!!!!!!!!
entity found!!!!!!!!!!!
[{'similarityScore': 0.9809149, 'uuid': '35825d6b-1777-4c2b-8a5f-73b49d6230b6', 'model': 'JCV_FACE_K25000', 'collection': '', 'key': '', 'metadata': [], 'createdAt': '2023-08-03T03:44:50Z', 'updatedAt': '2023-08-03T03:44:50Z', 'expiresAt': '3000-01-01T00:00:00Z'}]
!!!!!!!!!!!!!!!!!!!!!!!
{'model': 'JCV_FACE_K25000', 'count': 0, 'maxEntities': 1, 'threshold': 0.9, 'entitiesFound': [], 'entityUploaded': {'details': {'position': {'top': 420, 'left': 78, 'width': 456, 'height': 430}}}}
